In [11]:
import re

# Read the original file
with open("../src/aes_asm.S", "r") as file:
    lines = file.readlines()

# Dictionary to hold macro definitions
macros = {}

# Parse the file for macro definitions
new_lines = []
i = 0
while i < len(lines):
    if ".macro" in lines[i]:
        # Extract the macro name and arguments
        parts = lines[i].split()
        macro_name = parts[1]
        macro_args = [arg for part in parts[2:] for arg in part.split(',')] if len(parts) > 2 else []
        macro_args = [arg for arg in macro_args if arg != '']

        # Find the end of the macro definition
        end_index = i
        while ".endm" not in lines[end_index]:
            end_index += 1

        # Store the macro definition and arguments
        macros[macro_name] = (macro_args, lines[i + 1 : end_index])

        # Skip over the macro definition
        i = end_index + 1
    else:
        # Keep this line
        new_lines.append(lines[i])
        i += 1

lines = new_lines

# Expand the macros
for i in range(len(lines)):
    for macro in macros:
        if macro in lines[i]:
            # Extract the arguments from the macro invocation
            parts = lines[i].split()
            invocation_args = parts[1:] if len(parts) > 1 else []

            # Replace the arguments in the macro definition with the invocation arguments
            definition = [line for line in macros[macro][1]]
            for arg, value in zip(macros[macro][0], invocation_args):
                definition = [line.replace("\\"+arg, value) for line in definition]

            # Replace the macro invocation with its definition
            lines[i:i+1] = definition

# Write the result to a new file
with open("../src/aes_asm_expanded.S", "w") as file:
    file.writelines(lines)